# Linpackベンチマーク -- HPL -- Intel MLK Benchmark

---
OpenHPC環境で、[Intel MLK Benchmark Suite](https://software.intel.com/en-us/articles/intel-mkl-benchmarks-suite)に含まれているコンパイル済の[HPL](http://www.netlib.org/benchmark/hpl/)を実行します。

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。

* 構築済のOpenHPC環境がある
* OpenHPC環境のマスターノードに対してSSHでログインできる

> この Notebookでは、管理ユーザ以外が実行した状況に合わせるために Ansible ではなく ssh で操作を行います。

マスターノードに対して SSH でログインできることを確認します。マスターノードのアドレスを指定してください。

In [ ]:
# (例)
# master_address = '172.30.XXX.xxx'

master_address =

SSHでログインするユーザ名を指定してください。

In [ ]:
# (例)
# user = 'vcp'

user = 'vcp'

必要であればSSHの秘密鍵を指定してください。

In [ ]:
# (例)
# ssh_identity = '~/.ssh/id_rsa'

SSHでログインする際の引数を、変数に格納しておきます。

In [ ]:
# ユーザ名とホスト名
target = f'{user}@{master_address}'
print(target)

# SSHのコマンドライン引数
ssh_opts = f'-i {ssh_identity}' if 'ssh_identity' in vars() else ''
print(ssh_opts)

マスターノードに対してSSHでログインしてコマンドを実行してみます。

In [ ]:
!ssh {ssh_opts} {target} hostname

## 準備

HPLを実行するための準備作業を行います。

[Intel MLK Benchmark Suite](https://software.intel.com/en-us/articles/intel-mkl-benchmarks-suite)を取得して、ホームディレクトリに展開します。

**注意**

Intel MLK Benchmark Suiteをダウンロードすると[End User License Agreements](https://software.intel.com/en-us/articles/end-user-license-agreement)に同意したことになります。

In [ ]:
mlk_version = '2019.6.005'
mlk_url = f'https://software.intel.com/sites/default/files/managed/cc/19/l_mklb_p_{mlk_version}.tgz'

!ssh {ssh_opts} {target} curl -LO {mlk_url}
!ssh {ssh_opts} {target} tar xzf l_mklb_p_{mlk_version}.tgz

xhpl の実行用ディレクトリを作成します。

In [ ]:
hpl_dir = 'intel-hpl'
!ssh {ssh_opts} {target} mkdir -p {hpl_dir}

xhplコマンドのシンボリックリンクを作成します。

In [ ]:
!ssh {ssh_opts} {target} ln -sf \${{HOME}}/l_mklb_p_{mlk_version}/benchmarks_2019/linux/mkl/benchmarks/mp_linpack/xhpl_intel64_static {hpl_dir}/xhpl

## HPLの実行

HPLを実行します。

### パラメータの設定

計算ノードの数を指定してください。

In [ ]:
# (例)
# num_nodes = 2

num_nodes =

MPIのタスク数を指定してください。

In [ ]:
# (例)
# num_tasks = 4

num_tasks =

### HPL.dat の作成

HPL.dat を作成します。

`linux/mkl/benchmarks/mp_linpack/readme.txt` によると`NB`の推奨値は以下のようになっています。

```
Recommended blocking sizes (NB in HPL.dat) are listed below for various Intel(R)
architectures:

Intel(R) Xeon(R) Processor X56*/E56*/E7-*/E7*/X7*                             : 256
Intel(R) Xeon(R) Processor E26*/E26* v2                                       : 256
Intel(R) Xeon(R) Processor E26* v3/E26* v4                                    : 192
Intel(R) Core(TM) i3/5/7-6* Processor                                         : 192
Intel(R) Xeon Phi(TM) Processor 72*                                           : 336
Intel(R) Xeon(R) Scalabile Processors                                         : 384
```

次のセルの `N` や `NB` の値を必要に応じて書き換えて実行してください。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path
%run scripts/hpl_utils.py


p, q = hpl_pq(num_tasks)
with TemporaryDirectory() as workdir:
    hpl_dat = Path(workdir) / 'HPL.dat'
    with hpl_dat.open(mode='w') as f:
        f.write(f'''HPLinpack benchmark input file
Innovative Computing Laboratory, University of Tennessee
HPL.out      output file name (if any)
6            device out (6=stdout,7=stderr,file)
1            # of problems sizes (N)
10000        Ns
1            # of NBs
192          NBs
0            PMAP process mapping (0=Row-,1=Column-major)
1            # of process grids (P x Q)
{p}            Ps
{q}            Qs
16.0         threshold
1            # of panel fact
2            PFACTs (0=left, 1=Crout, 2=Right)
1            # of recursive stopping criterium
2            NBMINs (>= 1)
1            # of panels in recursion
2            NDIVs
1            # of recursive panel fact.
1            RFACTs (0=left, 1=Crout, 2=Right)
1            # of broadcast
0            BCASTs (0=1rg,1=1rM,2=2rg,3=2rM,4=Lng,5=LnM)
1            # of lookahead depth
0            DEPTHs (>=0)
0            SWAP (0=bin-exch,1=long,2=mix)
1            swapping threshold
1            L1 in (0=transposed,1=no-transposed) form
1            U  in (0=transposed,1=no-transposed) form
0            Equilibration (0=no,1=yes)
8            memory alignment in double (> 0)
''')
    !cat {hpl_dat}
    !scp {ssh_opts} {str(hpl_dat)} {target}:{hpl_dir}

### ジョブスクリプトの作成

HPLを実行するジョブスクリプトを作成します。

次の例ではOpen MPIの代わりにMPICHを使うスクリプトになっています。

In [ ]:
with TemporaryDirectory() as workdir:
    hpl_batch = Path(workdir) / 'hpl-intel.job'
    with hpl_batch.open(mode='w') as f:
        f.write(f'''#!/bin/bash

#SBATCH -J hpl-intel          # Job name
#SBATCH -o hpl-intel.%j.out   # Name of stdout output file (%j expands to jobId)
#SBATCH -N {num_nodes}                  # Total number of nodes requested
#SBATCH -n {num_tasks}                  # Total number of mpi tasks requested
#SBATCH -t 01:00:00           # Run time (hh:mm:ss)

# Launch MPI-based executable

if module is-loaded openmpi3; then
    module swap openmpi3 mpich
elif module is-loaded mvapich2; then
    module swap mvapich2 mpich
fi

cd {hpl_dir}

prun ./xhpl        
''')
    !cat {hpl_batch}
    !scp {ssh_opts} {str(hpl_batch)} {target}:.

### ジョブの実行

HPLのジョブを実行します。

In [ ]:
!ssh {ssh_opts} {target} sbatch hpl-intel.job

ジョブの状態を確認します。

In [ ]:
!ssh {ssh_opts} {target} squeue

結果を確認します。

> ジョブの実行が完了するまで数分程度かかります。しばらく待ってから次のセルを実行してください。

In [ ]:
!ssh {ssh_opts} {target} 'cat hpl-intel.*.out'